# Measuring quantum states in QuAIRKit

This tutorial demonstrates how to perform quantum measurement using QuAIRKit.

**Table of Contents**

- [Quantum measurement](#Quantum-measurement)
- [Quantum Circuit](#Quantum-circuit)
  - [Example](#Example) 
- [Measurement](#Measurement)

In [1]:

import time
import traceback

import torch
import quairkit as qkit
from quairkit.database import *
from quairkit.loss import Measure
from quairkit.qinfo import prob_sample

qkit.set_dtype('complex128')

## Quantum measurement

The idea of measurement comes from one of the four postulates in quantum machanics.

> Postulate 3 [1]
>
> *Quantum measurement*s are described by a collection $\{M_m\}_m$ of measurement operators. These are operators acting on the state space of the system being measured. The index $m$ refers to the measurement outcomes that may occur in the experiment. If the state of the quantum system is $|\psi\rangle$ immediately before the measurement, then the probability that result $m$ occurs is given by
> $$p(m) = \langle\psi|  M_m^\dagger M_m |\psi\rangle$$
> and the state of the system after the measurement is
> $$\frac{M_m |\psi\rangle}{\sqrt{\langle\psi| M_m^\dagger M_m |\psi\rangle}}$$
> 
> where the measurement operators satisfy the completeness equation,
> $$\sum_m M_m^\dagger M_m = I.$$

Such operator set $\{M_m^\dagger M_m\}_m$ is called a *Positive Operator-Valued Measure* (POVM). When all $M_m$ are orthogonal projectors  (i.e., $M_m = M_m^\dagger = M_m^2$), this set is called a *Projection-valued Measure* (PVM). The quantum measurement described by PVM is called a *projective measurement*. 

We usually perform projective measurements based on the eigenbasis of an observable. For example, we can generate a PVM for the Pauli string 'x'.

In [2]:
pvm = pauli_str_povm('x')
print(pvm)

tensor([[[ 0.5000+0.j,  0.5000+0.j],
         [ 0.5000+0.j,  0.5000+0.j]],

        [[ 0.5000+0.j, -0.5000+0.j],
         [-0.5000+0.j,  0.5000+0.j]]])


## Perform measurement

Projective measurements in QuAIRKit are mainly called by a torch Module `Measure`. There are three ways to initialize a `Measure` instance:

1. Set computational measurement by default, i.e., $M_m = \{|m\rangle\langle m|\}$
2. Set measurement by given pauli string(s)
3. Set measurement by given PVM(s) in torch.Tensor

For Measure instances initialized in the first two ways, if the measurement is across all qubits, then the output state(s) will always be recognized as pure state(s).

In [3]:
op = Measure() # computational measure
op = Measure('x') # x measure on a qubit
op = Measure(pvm) # measure with a pvm

Measure accepts the `State` instances and an (optional) measure position as input and returns the measurement result. Note that if measure only happens on a part of the system, then the argument `qubits_idx` should be specified.

In [4]:
num_qubits = 1
rho = random_state(num_qubits, rank=2)

prob = op(rho, qubits_idx=[0]) # measure rho
print('The probability distribution is stored in shape', prob.shape)

The probability distribution is stored in shape torch.Size([2])


Measure can retain the collapsed state after measurement by setting `keep_state = True`.

In [5]:
prob, collapsed_state = op(rho, keep_state=True)
print('The collapsed state for each outcome is', collapsed_state)

The collapsed state for each outcome is 
---------------------------------------------------
 Backend: density_matrix
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
 # 1:
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
---------------------------------------------------



Measure can only return the particular measurement outcome by setting `desired_result=x`

In [6]:
x = '1'
prob, collapsed_state = op(rho, keep_state=True, desired_result=x) # return the second outcome
print(f'The probability for obtaining outcome {x} is {prob}, with outcome state', collapsed_state)

The probability for obtaining outcome 1 is tensor([0.2544]), with outcome state 
---------------------------------------------------
 Backend: density_matrix
 System dimension: [2]
 System sequence: [0]
 Batch size: [1]

 # 0:
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
---------------------------------------------------



You can also directly call the attribute `measure` of `State` instances for simple measurement.

In [7]:
prob = rho.measure(pvm) # same as Measure(pvm)(rho)
prob, collapsed_state = rho.measure(pvm, keep_state=True) # same as Measure(pvm)(rho, keep_state=True)

### Positive operator-valued measurement

POVM is a generalization of PVM, yet it may be non-physical. In QuAIRKit, we can perform positive operator-valued measurement by  `State.measure`, with `is_povm` set to True.

Computation for POVM is often more efficient than that for PVM, as it directly computes the probability. However, its potentially lack of a unique post-measurement state makes it less useful in practice.

In [8]:
start_time = time.time()
prob = rho.measure(pvm)
print(f'Time for measuring with pvm: {time.time() - start_time:.10f}s')


start_time = time.time()
prob = rho.measure(pvm, is_povm=True)
print(f'Time for measuring with povm: {time.time() - start_time:.10f}s')

try:
    rho.measure(pvm, is_povm=True, keep_state=True)
except ValueError:
    traceback.print_exc()

Time for measuring with pvm: 0.0005207062s
Time for measuring with povm: 0.0003004074s


Traceback (most recent call last):
  File "/tmp/ipykernel_70098/889726792.py", line 11, in <module>
    rho.measure(pvm, is_povm=True, keep_state=True)
  File "/home/leiz/QuAIRKit-Dev/quairkit/core/state/backend/__init__.py", line 604, in measure
    raise ValueError(
ValueError: `is_povm` and `keep_state` cannot be both True, since a general POVM does not distinguish states.


## Batch Measurement

QuAIRKit supports batched measurement under broadcasting rule, either one or multiple PVMs, or one or multiple input states, or both. The broadcasting rule is summarized as follows:

|    PVM of size m    |     State      |  Probability       |
|:----------------:|:---------------------:|:---------------------:|
| [None, ...]   | [None, ...] | [m] |
| [None, ...] | [n, ...]       | [n, m] |
| [n, ...] | [None, ...]       | [n, m] |
| [n, ...] | [n, ...]       | [n, m] |
| [n, ...] | [p, ...]       | Error |

Here the first dimension indicates the batch size of PVMs and input states.

You can initialize a batch Measure instance via Pauli string, or directly input the batched PVM in torch.Tensor.

In [9]:
batch_size = 3

op = Measure(['x', 'y', 'z']) # measure states by x, y, z basis, respectively

list_pvm = pauli_str_povm(['x', 'y', 'z'])
print(list_pvm.shape)
op = Measure(list_pvm) # equivalent to Measure(['x', 'y', 'z'])

torch.Size([3, 2, 2, 2])


As shown by above table, you can apply `Measure` to a single state

In [10]:
prob, collapsed_state = op(rho, keep_state=True)
print('The measured states for the first batch is', collapsed_state[0],
      f'with prob distribution {prob[0]}')

The measured states for the first batch is 
---------------------------------------------------
 Backend: density_matrix
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
 # 1:
[[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
---------------------------------------------------
 with prob distribution tensor([0.7456, 0.2544])


or apply `Measure` to a batched state. Note that the batch dimension need to be matched.

In [11]:
list_rho = random_state(num_qubits, size=batch_size)

prob, collapsed_state = op(list_rho, keep_state=True)
print('The measured states for the first batch is', collapsed_state[0],
      f'with prob distribution {prob[0]}')

The measured states for the first batch is 
---------------------------------------------------
 Backend: state_vector
 System dimension: [2]
 System sequence: [0]
 Batch size: [2]

 # 0:
[0.58-0.4j 0.58-0.4j]
 # 1:
[-0.06+0.7j  0.06-0.7j]
---------------------------------------------------
 with prob distribution tensor([0.9872, 0.0128])


### Sampled measurements

You can use `quairkit.qinfo.prob_sample` to generate shots of qubits measured based on given probability distributions.

For example, in 1024 shots, `'00': tensor([98, ...])` indicates 98 occurrences for measuring 00 in the first probability distribution, and so on. You can also adjust the argument `binary` and `proportional` to change the output format:

- `binary` is False: the dictionary index is in the decimal system. 
- `proportional` is True: values are transformed into proportions.

In [12]:
print(f'{batch_size} probability distributions are\n', prob)

print(prob_sample(prob))
print(prob_sample(prob, binary=False))
print(prob_sample(prob, proportional=True))

3 probability distributions are
 tensor([[0.9872, 0.0128],
        [0.8805, 0.1195],
        [0.8752, 0.1248]])
{'0': tensor([1018,  893,  903]), '1': tensor([  6, 131, 121])}
{'0': tensor([1019,  893,  884]), '1': tensor([  5, 131, 140])}
{'0': tensor([0.9932, 0.8848, 0.8633]), '1': tensor([0.0068, 0.1152, 0.1367])}


---

## References

[1] Nielsen, Michael A., and Isaac L. Chuang. Quantum computation and quantum information. Vol. 2. Cambridge: Cambridge university press, 2001.